In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from utils.metrics import performance

In [16]:
test_dir = 'dataset/Kather_texture_2016_image_tiles_5000/test'

In [17]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [18]:
# Define image parameters
image_size = (150, 150)
batch_size = 32
num_classes = 8

In [19]:
# Evaluate the model
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=1,
    class_mode='categorical',  # For multiclass classification
    shuffle=False
)

Found 504 images belonging to 8 classes.


In [22]:
alexnet = tf.keras.models.load_model('savedModels/AlexNet.keras')
googlenet = tf.keras.models.load_model('savedModels/GoogleNet.keras')
resnet = tf.keras.models.load_model('savedModels/ResNet.keras')
mobilenet = tf.keras.models.load_model('savedModels/MobileNet.keras')
squeezenet = tf.keras.models.load_model('savedModels/SqueezeNet.keras')

/Users/karthik/Desktop/UROP/Code/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 18 variables whereas the saved optimizer has 34 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
/Users/karthik/Desktop/UROP/Code/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 172 variables whereas the saved optimizer has 10 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [27]:
models = [alexnet, googlenet, resnet, mobilenet, squeezenet]

In [92]:
dataset = []

In [93]:
for i in range(len(test_generator)):
    data = []
    for model in models:
        output = model.predict(test_generator[i][0])[0]
        data.extend(output)
    data.append(np.argmax(test_generator[i][1]))
    dataset.append(data)
        

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

In [40]:
import pandas as pd

In [95]:
df = pd.DataFrame(data=dataset)

In [96]:
df.to_csv('merge.csv', index=False)

Code

In [120]:
df_new = pd.read_csv('./merge.csv')

In [122]:
data = df_new.iloc[:,:40]

In [126]:
labels = df_new.iloc[:,-1]

In [131]:
from sklearn.model_selection import train_test_split


In [132]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [143]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input

# Assuming you have already split your data into X_train, X_test, y_train, y_test

# Define the model
model = Sequential([
    Input(shape=(40,)),
    Dense(64, activation='relu'),  # Input layer with 40 input nodes
    Dense(32, activation='relu'),  # Hidden layer with 32 neurons and ReLU activation
    Dense(8, activation='softmax')  # Output layer with 8 neurons (one for each class) and softmax activation
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)  # Example values, adjust as needed

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1503 - loss: 2.0453 - val_accuracy: 0.2963 - val_loss: 1.8550
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2729 - loss: 1.8136 - val_accuracy: 0.6173 - val_loss: 1.6656
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6360 - loss: 1.6025 - val_accuracy: 0.7901 - val_loss: 1.4808
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8297 - loss: 1.3829 - val_accuracy: 0.8765 - val_loss: 1.2801
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9157 - loss: 1.1562 - val_accuracy: 0.9012 - val_loss: 1.0702
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9380 - loss: 0.9235 - val_accuracy: 0.9136 - val_loss: 0.8629
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9265 - loss: 0.7307 - val_accuracy: 0.9259 - val_loss: 0.6845
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9312 - loss: 0.5507 - val_accuracy: 0.9259 - val_loss

In [144]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Assuming you have already trained your model and evaluated it on the test set
# model.evaluate() should have been called and test predictions stored in test_predictions

# Generate predictions on the test set
test_predictions = model.predict(X_test)
# Convert predicted probabilities to class labels
predicted_classes = np.argmax(test_predictions, axis=1)

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_test, predicted_classes)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Confusion Matrix:
[[ 7  0  4  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0]
 [ 1  1  6  0  0  0  0  0]
 [ 0  1  0  9  0  0  0  0]
 [ 0  2  0  0 10  0  0  0]
 [ 1  0  0  1  0  9  0  0]
 [ 0  0  0  0  0  0  9  1]
 [ 0  0  0  0  0  0  1 20]]
